<a href="https://colab.research.google.com/github/mallibus/TF2---colab/blob/master/Colab_6_Transfer_Learning_and_Fine_Tunning_TF2_vs_fastai_on_RESNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning con Tensorflow - Resnet50 su dataset PETS fast.ai


In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

In [2]:
!pip install tqdm

In [0]:
from pathlib import Path

In [4]:
!pip install fastai


In [0]:
#importo anche fastai per scaricare il dataset 
from fastai.vision import *

In [6]:
path = untar_data(URLs.PETS); path

PosixPath('/root/.fastai/data/oxford-iiit-pet')

In [7]:
print(path.ls())
path_anno = path/'annotations'
path_img = path/'images'

[PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images')]


In [8]:
fnames = get_image_files(path_img)
fnames[:5]

[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/leonberger_155.jpg'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/images/havanese_187.jpg'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/images/saint_bernard_50.jpg'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Ragdoll_104.jpg'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/images/saint_bernard_190.jpg')]

## Stage 2: Dataset preprocessing

### Import project dependencies

In [9]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0-alpha0'

### Unzipping the Dogs vs Cats dataset

In [10]:
path_anno.ls()

[PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations/trainval.txt'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations/test.txt'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations/._trimaps'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations/xmls'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations/list.txt'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations/trimaps'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations/README')]

In [11]:
with open(path_anno/'README') as f:
  lines = f.readlines()
for l in lines[:100]:
  print(l,end='')

OXFORD-IIIT PET Dataset
-----------------------
Omkar M Parkhi, Andrea Vedaldi, Andrew Zisserman and C. V. Jawahar

We have created a 37 category pet dataset with roughly 200 images for each class. 
The images have a large variations in scale, pose and lighting. All images have an 
associated ground truth annotation of breed, head ROI, and pixel
level trimap segmentation.

Contents:
--------
trimaps/ 	Trimap annotations for every image in the dataset
		Pixel Annotations: 1: Foreground 2:Background 3: Not classified
xmls/		Head bounding box annotations in PASCAL VOC Format

list.txt	Combined list of all images in the dataset
		Each entry in the file is of following nature:
		Image CLASS-ID SPECIES BREED ID
		ID: 1:37 Class ids
		SPECIES: 1:Cat 2:Dog
		BREED ID: 1-25:Cat 1:12:Dog
		All images with 1st letter as captial are cat images while
		images with small first letter are dog images.
trainval.txt	Files describing splits used in the paper.However,
test.txt	you are encouraged to try ra

In [12]:
with open(path_anno/'list.txt') as f:
  lines = f.readlines()
lines = np.array(lines)
for l in np.random.choice(lines,size=20,replace=False):
  print(l,end='')

japanese_chin_177 18 2 12
english_setter_75 14 2 8
wheaten_terrier_60 36 2 24
havanese_133 17 2 11
boxer_18 9 2 5
havanese_23 17 2 11
boxer_197 9 2 5
saint_bernard_19 29 2 19
Russian_Blue_177 28 1 10
Egyptian_Mau_122 12 1 6
Bombay_156 8 1 4
american_pit_bull_terrier_87 3 2 2
american_bulldog_133 2 2 1
saint_bernard_171 29 2 19
staffordshire_bull_terrier_112 35 2 23
keeshond_48 19 2 13
scottish_terrier_71 31 2 21
Birman_199 7 1 3
Egyptian_Mau_34 12 1 6
english_cocker_spaniel_107 13 2 7


In [13]:
for l in lines[:10]:
  print(l,end='')

#Image CLASS-ID SPECIES BREED ID
#ID: 1:37 Class ids
#SPECIES: 1:Cat 2:Dog
#BREED ID: 1-25:Cat 1:12:Dog
#All images with 1st letter as captial are cat images
#images with small first letter are dog images
Abyssinian_100 1 1 1
Abyssinian_101 1 1 1
Abyssinian_102 1 1 1
Abyssinian_103 1 1 1


In [0]:
list_for_df = []
for l in lines[6:]:
  list_for_df.append({'filename': f'{l[:-1].split()[0]}.jpg',
                      'class_name': "_".join(l[:-1].split()[0].split("_")[:-1]),
                      'class_number' : l[:-1].split()[1]})

In [0]:
dataset_df = pd.DataFrame(list_for_df)

In [16]:
dataset_df.head()

,class_name,class_number,filename
0,Abyssinian,1,Abyssinian_100.jpg
1,Abyssinian,1,Abyssinian_101.jpg
2,Abyssinian,1,Abyssinian_102.jpg
3,Abyssinian,1,Abyssinian_103.jpg
4,Abyssinian,1,Abyssinian_104.jpg


### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [0]:
data_gen = ImageDataGenerator(rescale=1/255.,validation_split=0.2,
                              #zoom_range= [.15, .15],fill_mode='wrap',
                              #horizontal_flip=True,
                              #width_shift_range=0.1,height_shift_range=0.1,
                              #rotation_range=10
                             )


In [18]:
train_generator = data_gen.flow_from_dataframe(dataset_df,path_img, 
                                                     x_col='filename',y_col='class_name', 
                                                     target_size=(224,224), batch_size=16,
                                                     subset='training',shuffle=True,
                                                     class_mode="categorical")
valid_generator = data_gen.flow_from_dataframe(dataset_df,path_img, 
                                                     x_col='filename',y_col='class_name', 
                                                     target_size=(224,224), batch_size=16,
                                                     subset='validation',shuffle=True,
                                                     class_mode="categorical")



Found 5880 validated image filenames belonging to 37 classes.
Found 1469 validated image filenames belonging to 37 classes.


## Building the model

### Loading the pre-trained model (Resnet50)

In [0]:
IMG_SHAPE = (224, 224, 3)

In [20]:
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [21]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

### Freezing the base model

In [0]:
base_model.trainable = False

### Defining the custom head for our network

Come da corso TF2.0 di Udemy, uso un GlobalAveragePooling
non mi ricordo cosa usava Geremy come ultimi livelli... controllerò

In [23]:
base_model.output

<tf.Tensor 'activation_48/Relu:0' shape=(None, 7, 7, 2048) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [25]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 2048) dtype=float32>

In [26]:
output_nodes = len(train_generator.class_indices)
output_nodes

37

In [0]:
prediction_layer = tf.keras.layers.Dense(units=output_nodes, activation='softmax')(global_average_layer)

### Defining the model

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                      
______________________________________________________________________________________________

### Compiling the model

In [0]:

model.compile(tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

### Training the model

In [31]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)


Epoch 1/5
367/367 [==============================] - 129s 350ms/step - loss: 0.9959 - accuracy: 0.7113 - val_loss: 3.7795 - val_accuracy: 0.0144
Epoch 2/5
367/367 [==============================] - 91s 248ms/step - loss: 0.2732 - accuracy: 0.9134 - val_loss: 2.6835 - val_accuracy: 0.3345
Epoch 3/5
367/367 [==============================] - 91s 247ms/step - loss: 0.1402 - accuracy: 0.9580 - val_loss: 2.4234 - val_accuracy: 0.7328
Epoch 4/5
367/367 [==============================] - 91s 247ms/step - loss: 0.1126 - accuracy: 0.9678 - val_loss: 0.9639 - val_accuracy: 0.8098
Epoch 5/5
367/367 [==============================] - 91s 247ms/step - loss: 0.0921 - accuracy: 0.9722 - val_loss: 2.8750 - val_accuracy: 0.7060


### Transfer learning model evaluation

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [34]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.7059224247932434


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

anche i livelli da sbloccare sono scelti un po' a caso...

In [0]:
base_model.trainable = True

In [36]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 175


In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [0]:


model.compile(tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

### Fine tuning

In [40]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
368/368 [==============================] - 78s 213ms/step - loss: 0.0427 - accuracy: 0.9854 - val_loss: 1.1594 - val_accuracy: 0.7992
Epoch 2/5
368/368 [==============================] - 55s 150ms/step - loss: 0.0354 - accuracy: 0.9908 - val_loss: 1.2997 - val_accuracy: 0.8128
Epoch 3/5
368/368 [==============================] - 55s 149ms/step - loss: 0.0259 - accuracy: 0.9929 - val_loss: 0.9917 - val_accuracy: 0.8155
Epoch 4/5
368/368 [==============================] - 55s 149ms/step - loss: 0.0283 - accuracy: 0.9920 - val_loss: 1.2868 - val_accuracy: 0.8012
Epoch 5/5
368/368 [==============================] - 55s 150ms/step - loss: 0.0233 - accuracy: 0.9935 - val_loss: 1.3260 - val_accuracy: 0.8094


### Evaluating the fine tuned model

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [42]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.809394121170044


In [0]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
368/368 [==============================] - 54s 147ms/step - loss: 0.0215 - accuracy: 0.9930 - val_loss: 1.3790 - val_accuracy: 0.8074
Epoch 2/5
266/368 [====================>.........] - ETA: 13s - loss: 0.0131 - accuracy: 0.9969